In [ ]:
# Install required dependencies
!pip install -q gradio PyPDF2 python-docx nest_asyncio requests "pydantic==2.5.0" langchain_community "crewai[tools]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.2/471.2 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.11
    Uninstalling langchain-text-splitters-0.3.11:
      Successfully uninstalled langchain-text-splitters-0.3.11
ERROR: pip's dependency resolver

In [ ]:
# Environment setup and dependency bootstrap
import os
import sys
import json
import time
import re
import asyncio
import requests
import traceback
import subprocess
from io import BytesIO
from pathlib import Path
from typing import List, Dict, Any

# CRITICAL: Set multiple dummy keys BEFORE importing CrewAI
os.environ["OPENAI_API_KEY"] = "sk-dummy-local-only"
os.environ["OPENAI_API_BASE"] = "http://localhost:11434"
os.environ["LITELLM_LOG"] = "ERROR"  # Suppress LiteLLM logs

# -----------------------------
# Install/Import dependencies with specific versions
# -----------------------------
def _ensure(pkg, import_name=None, version=None):
    try:
        __import__(import_name or pkg.split('[')[0])
    except Exception:
        install_pkg = f"{pkg}=={version}" if version else pkg
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", install_pkg])
        __import__(import_name or pkg.split('[')[0])

print("📦 Installing dependencies...")
_ensure("gradio")
_ensure("PyPDF2", "PyPDF2")
_ensure("python-docx", "docx")
_ensure("nest_asyncio")
_ensure("requests")
_ensure("pydantic", version="2.5.0")  # Specific version for compatibility
_ensure("langchain_community")
_ensure("crewai[tools]")  # Install with tools support

In [ ]:
# Imports after dependency bootstrap
import gradio as gr
import PyPDF2
import docx
import nest_asyncio
from pydantic import BaseModel, Field

# Import CrewAI components AFTER environment setup
try:
    from crewai import Agent, Task, Crew, Process
    from crewai.tools import tool
    from langchain_community.llms import Ollama as LC_Ollama
except ImportError as e:
    print(f"❌ CrewAI import error: {e}")
    print("Reinstalling crewai...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "--force-reinstall", "crewai[tools]"])
    from crewai import Agent, Task, Crew, Process
    from crewai.tools import tool
    from langchain_community.llms import Ollama as LC_Ollama

In [ ]:
# Configuration
OLLAMA_BASE = os.environ.get("OLLAMA_BASE", "http://127.0.0.1:11434")
DEFAULT_MODEL = os.environ.get("OLLAMA_MODEL", "llama3.1:8b-instruct-q4_K_M")

MODEL_CHOICES = [
    "llama3.1:8b-instruct-q4_K_M",
    "mistral:7b-instruct-v0.3-q5_0",
    "llama3.1:8b-instruct",
    "mistral:7b-instruct",
    "gpt-oss:20b",
]

MAX_INPUT_TOKENS    = 2000
MAX_OUTPUT_TOKENS   = 800
MAX_QUESTION_TOKENS = 600
MAX_CONTEXT_TOKENS  = 1200

CURRENT_CONTEXT = ""
LAST_ANALYSIS = {"jd": "", "insights": ""}

CHAT_AVAILABLE = None
ACTIVE_MODEL   = None

In [ ]:
# System helpers (Ollama daemon + model)
def _run(cmd, check=True, env=None):
    return subprocess.run(cmd, check=check, text=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, env=env)

def _binary_exists(name):
    try:
        _run(["bash", "-lc", f"command -v {name}"])
        return True
    except Exception:
        return False

def _install_ollama_cli():
    _run(["bash", "-lc", "curl -fsSL https://ollama.com/install.sh | sh"], check=True)

def _start_ollama_daemon():
    env = os.environ.copy()
    _run(["bash", "-lc", "nohup ollama serve > /tmp/ollama.log 2>&1 &"], check=False, env=env)

def _wait_for_http(url, timeout=180, interval=3):
    start = time.time()
    last_err = None
    while time.time() - start < timeout:
        try:
            r = requests.get(url, timeout=5)
            if r.status_code == 200:
                return True
        except Exception as e:
            last_err = e
        time.sleep(interval)
    if last_err:
        raise RuntimeError(f"Timeout reaching {url}: {last_err}")
    return False

def _list_models():
    try:
        r = requests.get(f"{OLLAMA_BASE}/api/tags", timeout=10)
        r.raise_for_status()
        data = r.json()
        return [m.get("name") for m in data.get("models", []) if m.get("name")]
    except Exception:
        return []

def _pull_model_blocking(tag):
    try:
        print(f"⬇️ Pulling model: {tag}")
        env = os.environ.copy()
        res = _run(["bash", "-lc", f"ollama pull {tag}"], check=True, env=env)
        print(res.stdout.strip())
        return True
    except subprocess.CalledProcessError as e:
        print(f"Pull failed for {tag}:\n{e.stdout}")
        return False
    except Exception as e:
        print(f"Pull exception for {tag}: {e}")
        return False

def _resolve_installed_tag(preferred):
    models = _list_models()
    if not models:
        return None
    if preferred in models:
        return preferred
    for cand in [
        "llama3.1:8b-instruct-q4_K_M",
        "mistral:7b-instruct-v0.3-q5_0",
        "llama3.1:8b-instruct",
        "mistral:7b-instruct",
    ]:
        if cand in models:
            return cand
    return models[0]

def _ensure_ollama_ready():
    try:
        r = requests.get(f"{OLLAMA_BASE}/api/tags", timeout=5)
        if r.ok:
            return
        raise RuntimeError(f"Ollama API status {r.status_code}")
    except Exception:
        if not _binary_exists("ollama"):
            _install_ollama_cli()
        _start_ollama_daemon()
        _wait_for_http(f"{OLLAMA_BASE}/api/tags", timeout=180, interval=3)

def _ensure_model_available(tag):
    global ACTIVE_MODEL
    models = _list_models()
    if tag not in models:
        if not _pull_model_blocking(tag):
            ACTIVE_MODEL = _resolve_installed_tag(tag)
            return
    ACTIVE_MODEL = tag

def _probe_chat_support():
    global CHAT_AVAILABLE
    try:
        payload = {"model": ACTIVE_MODEL, "messages": [{"role": "user", "content": "hi"}], "stream": False}
        resp = requests.post(f"{OLLAMA_BASE}/api/chat", json=payload, timeout=15)
        if resp.status_code == 404:
            CHAT_AVAILABLE = False
        else:
            resp.raise_for_status()
            CHAT_AVAILABLE = True
    except requests.HTTPError as he:
        if getattr(he.response, "status_code", None) == 404:
            CHAT_AVAILABLE = False
        else:
            CHAT_AVAILABLE = True
    except Exception:
        CHAT_AVAILABLE = False

In [ ]:
# Token helpers
def limit_tokens_by_words(text: str, max_tokens: int) -> str:
    if not text or not text.strip():
        return text
    words = text.split()
    if len(words) > max_tokens:
        return " ".join(words[:max_tokens]) + f"\n\n[TRUNCATED: kept first {max_tokens} tokens]"
    return text

def limit_input(text: str, cap: int) -> str:
    return limit_tokens_by_words(text, cap)

def limit_output(text: str, cap: int) -> str:
    return limit_tokens_by_words(text, cap)

In [ ]:
# File extraction
def _extract_pdf_text_from_bytes(pdf_bytes: bytes) -> str:
    try:
        reader = PyPDF2.PdfReader(BytesIO(pdf_bytes))
        if getattr(reader, "is_encrypted", False):
            return "❌ ERROR: PDF is password protected."
        text = []
        for i, page in enumerate(reader.pages):
            try:
                t = page.extract_text() or ""
                text.append(t)
            except Exception:
                return f"❌ ERROR: Could not read page {i+1} of PDF."
        final = "\n".join(text).strip()
        if not final:
            return "❌ ERROR: PDF appears empty or image-only."
        return final
    except Exception as e:
        return f"❌ ERROR: Invalid PDF. Details: {e}"

def _extract_docx_text(path: str) -> str:
    try:
        d = docx.Document(path)
        parts = []
        for p in d.paragraphs:
            if p.text.strip():
                parts.append(p.text)
        for table in d.tables:
            for row in table.rows:
                for cell in row.cells:
                    if cell.text.strip():
                        parts.append(cell.text)
        text = "\n".join(parts).strip()
        if not text:
            return "❌ ERROR: DOCX appears empty."
        return text
    except Exception as e:
        return f"❌ ERROR: Cannot read DOCX. Details: {e}"

def _extract_txt_text(path: str) -> str:
    encs = ["utf-8","latin-1","cp1252","iso-8859-1"]
    for enc in encs:
        try:
            with open(path, "r", encoding=enc) as f:
                txt = f.read().strip()
                if txt:
                    return txt
        except UnicodeError:
            continue
        except Exception as e:
            return f"❌ ERROR: Cannot read TXT. Details: {e}"
    return "❌ ERROR: Unsupported TXT encoding."

def extract_text_from_uploaded_file(file_obj) -> str:
    try:
        path = getattr(file_obj, "name", None) or (file_obj if isinstance(file_obj, str) else None)
        if not path or not os.path.exists(path):
            return "❌ ERROR: File not found."
        size = os.path.getsize(path)
        if size == 0:
            return "❌ ERROR: File is empty."
        if size > 10 * 1024 * 1024:
            return "❌ ERROR: File too large (>10MB)."
        ext = Path(path).suffix.lower()
        if ext == ".pdf":
            with open(path, "rb") as f:
                return _extract_pdf_text_from_bytes(f.read())
        if ext == ".docx":
            return _extract_docx_text(path)
        if ext == ".txt":
            return _extract_txt_text(path)
        return f"❌ ERROR: Unsupported file type '{ext}'. Use PDF/DOCX/TXT."
    except Exception as e:
        return f"❌ ERROR: Failed to process file. Details: {e}"

In [ ]:
# Scoring helper
def extract_match_score(text: str) -> int:
    patterns = [
        r"OVERALL MATCH SCORE:\s*\**(\d+)\**/100",
        r"MATCH SCORE:\s*\**(\d+)\**/100",
        r"SCORE:\s*\**(\d+)\**/100",
        r"(\d+)/100"
    ]
    for p in patterns:
        m = re.search(p, text, flags=re.IGNORECASE)
        if m:
            try:
                n = int(m.group(1))
                return max(0, min(100, n))
            except:
                pass
    return 50

In [ ]:
# Ollama API helpers
def _messages_to_prompt(messages):
    parts = []
    sys_msgs = [m["content"] for m in messages if m.get("role") == "system"]
    if sys_msgs:
        parts.append("System:\n" + "\n".join(sys_msgs).strip())
    for m in messages:
        role = m.get("role")
        if role in ("user", "assistant"):
            parts.append(f"{role.capitalize()}:\n{m.get('content','').strip()}")
    parts.append("Assistant:")
    return "\n\n".join(parts).strip()

def ollama_generate(prompt, temperature=0.2, max_tokens=MAX_OUTPUT_TOKENS):
    payload = {
        "model": ACTIVE_MODEL,
        "prompt": prompt,
        "stream": False,
        "options": {"temperature": float(temperature), "num_predict": int(max_tokens)}
    }
    resp = requests.post(f"{OLLAMA_BASE}/api/generate", json=payload, timeout=120)
    if resp.status_code == 404:
        raise requests.HTTPError("404 on /api/generate", response=resp)
    resp.raise_for_status()
    data = resp.json()
    if isinstance(data, dict) and "response" in data:
        return data["response"]
    return json.dumps(data, indent=2)

def ollama_chat(messages, temperature=0.2, max_tokens=MAX_OUTPUT_TOKENS):
    global CHAT_AVAILABLE
    if CHAT_AVAILABLE is None:
        _probe_chat_support()
    if CHAT_AVAILABLE is False:
        prompt = _messages_to_prompt(messages)
        return ollama_generate(prompt, temperature=temperature, max_tokens=max_tokens)
    payload = {
        "model": ACTIVE_MODEL,
        "messages": messages,
        "stream": False,
        "options": {"temperature": float(temperature), "num_predict": int(max_tokens)}
    }
    resp = requests.post(f"{OLLAMA_BASE}/api/chat", json=payload, timeout=120)
    if resp.status_code == 404:
        CHAT_AVAILABLE = False
        prompt = _messages_to_prompt(messages)
        return ollama_generate(prompt, temperature=temperature, max_tokens=max_tokens)
    resp.raise_for_status()
    data = resp.json()
    if isinstance(data, dict) and "message" in data and isinstance(data["message"], dict) and "content" in data["message"]:
        return data["message"]["content"]
    if isinstance(data, dict) and "response" in data:
        return data["response"]
    return json.dumps(data, indent=2)

In [ ]:
# Prompts for analysis
def make_jd_prompt(resume_text: str, jd_text: str) -> list:
    sys_prompt = (
        "You are a brutally honest recruitment analyst. Analyze the resume against the job description and "
        "produce a realistic, strict evaluation with clear scores and rationale."
    )
    user_prompt = f"""
RESUME:
{resume_text}

JOB DESCRIPTION:
{jd_text}

Provide response in EXACTLY this format:

**OVERALL MATCH SCORE: [X]/100** [star rating: ⭐ for <30, ⭐⭐ for 30-50, ⭐⭐⭐ for 50-70, ⭐⭐⭐⭐ for 70-85, ⭐⭐⭐⭐⭐ for 85+]

**TECHNICAL SKILLS ([X]/10):**
✅ Matched: [list]
❌ Missing: [list]

**EXPERIENCE LEVEL ([X]/10):**
• Years: [actual years]
• Required: [years required]
• Gap: [explain]

**EDUCATION ([X]/10):**
• Match Level: [Perfect/Good/Fair/Poor]

**SUMMARY:**
[Short honest rationale. If junior applying to senior, score low. Skills mismatch -> low score.]
""".strip()
    return [{"role": "system", "content": sys_prompt}, {"role": "user", "content": user_prompt}]

def make_insight_prompt(resume_text: str, jd_result: str) -> list:
    sys_prompt = "You are a logical talent assessor. Your recommendation MUST be consistent with the JD match score."
    user_prompt = f"""
RESUME:
{resume_text}

JD MATCH RESULT:
{jd_result}

CRITICAL RULE: Your recommendation MUST match the score:
- 0-30: HARD PASS
- 31-50: MAYBE (major concerns)
- 51-70: CAUTIOUS YES
- 71-85: RECOMMEND
- 86-100: STRONG RECOMMEND

Provide:
**RECOMMENDATION:** [HARD PASS/MAYBE/CAUTIOUS YES/RECOMMEND/STRONG RECOMMEND] - [one-line reason]
**KEY RISKS:** [bulleted list]
**INTERVIEW FOCUS:** [bulleted list]
""".strip()
    return [{"role": "system", "content": sys_prompt}, {"role": "user", "content": user_prompt}]

def make_hr_chat_prompt(question: str, context: str) -> list:
    sys_prompt = "You are an HR assistant who ONLY discusses the analyzed candidate. Refuse to answer any unrelated questions."
    user_prompt = f"""
CANDIDATE ANALYSIS CONTEXT:
{context}

HR QUESTION:
{question}

RULES:
1) ONLY answer about THIS candidate
2) Stay consistent with analysis scores
3) If match score is low (0-30), do not recommend hiring
4) If there are experience gaps, acknowledge them
5) NEVER answer general knowledge or unrelated questions
""".strip()
    return [{"role": "system", "content": sys_prompt}, {"role": "user", "content": user_prompt}]

In [ ]:
# CrewAI Tool Definitions (FIXED)
@tool("JD Match Analyzer")
def jd_match_tool_crewai(resume_text: str, job_description: str) -> str:
    """Analyzes resume against job description with brutal honesty. Returns compatibility score and detailed breakdown."""
    try:
        r = limit_input(resume_text, MAX_INPUT_TOKENS)
        j = limit_input(job_description, MAX_INPUT_TOKENS)
        out = ollama_chat(make_jd_prompt(r, j), temperature=0.2, max_tokens=MAX_OUTPUT_TOKENS)
        return limit_output(out, MAX_OUTPUT_TOKENS)
    except Exception as e:
        return f"ERROR: {str(e)}"

@tool("Critical Insight Generator")
def critical_insight_tool_crewai(resume_text: str, jd_match_result: str) -> str:
    """Generates critical insights consistent with JD match score. Returns recommendation aligned with score."""
    try:
        r = limit_input(resume_text, MAX_INPUT_TOKENS)
        d = limit_input(jd_match_result, MAX_INPUT_TOKENS)
        out = ollama_chat(make_insight_prompt(r, d), temperature=0.2, max_tokens=MAX_OUTPUT_TOKENS)
        return limit_output(out, MAX_OUTPUT_TOKENS)
    except Exception as e:
        return f"ERROR: {str(e)}"

@tool("HR Chat Assistant")
def hr_chat_tool_crewai(question: str, context: str) -> str:
    """Answers HR questions about the analyzed candidate ONLY. Refuses unrelated questions."""
    try:
        if not is_candidate_related_question(question):
            return (
                "🚫 I can only answer questions about the analyzed candidate.\n\n"
                "Please ask about hiring recommendation, strengths/weaknesses, score rationale, interview focus, or training needs."
            )
        q = limit_input(question, MAX_QUESTION_TOKENS)
        c = limit_input(context, MAX_CONTEXT_TOKENS)
        out = ollama_chat(make_hr_chat_prompt(q, c), temperature=0.2, max_tokens=MAX_OUTPUT_TOKENS)
        return limit_output(out, MAX_OUTPUT_TOKENS)
    except Exception as e:
        return f"ERROR: {str(e)}"

def is_candidate_related_question(question: str) -> bool:
    q = question.lower().strip()
    unrelated = ["weather","recipe","cooking","food","sports","politics","religion","celebrity","movie","music","game","animal","planet","element","chemistry","physics","math","history","geography","joke","story","define","explain","tell me about"]
    if any(k in q for k in unrelated):
        return False
    related = ["candidate","resume","hire","interview","skill","experience","qualification","score","match","recommend","suitable","fit","strength","weakness","concern","red flag","education","background","position","role","job","applicant","talent","employee","work","career","professional","technical","training","development"]
    if any(k in q for k in related):
        return True
    return True

In [ ]:
# CrewAI LLM and Agents (FIXED)
def get_crewai_llm():
    """Returns LangChain Ollama LLM configured for current model"""
    model_tag = ACTIVE_MODEL or DEFAULT_MODEL
    return LC_Ollama(
        base_url=OLLAMA_BASE,
        model=model_tag,
        temperature=0.2,
        num_predict=MAX_OUTPUT_TOKENS
    )

def build_crewai_agents():
    """Build CrewAI agents with proper LLM binding"""
    crew_llm = get_crewai_llm()

    jd_agent = Agent(
        role="Brutally Honest Job Matching Specialist",
        goal="Provide realistic compatibility scores with no sugar-coating",
        backstory="A no-nonsense recruitment analyst who gives honest scores.",
        verbose=False,
        allow_delegation=False,
        llm=crew_llm,
        tools=[jd_match_tool_crewai]
    )

    insight_agent = Agent(
        role="Logically Consistent Career Assessor",
        goal="Generate insights that match the compatibility scores with no contradictions",
        backstory="A logical talent assessor ensuring recommendations align with scores.",
        verbose=False,
        allow_delegation=False,
        llm=crew_llm,
        tools=[critical_insight_tool_crewai]
    )

    hr_agent = Agent(
        role="Focused HR Assistant (Candidate-only)",
        goal="ONLY answer questions about the analyzed candidate; refuse unrelated questions",
        backstory="A focused HR consultant who only discusses the analyzed candidate.",
        verbose=False,
        allow_delegation=False,
        llm=crew_llm,
        tools=[hr_chat_tool_crewai]
    )

    return jd_agent, insight_agent, hr_agent

In [ ]:
# Task Execution (FIXED)
def crew_run_jd(jd_agent, resume_text, job_description):
    """Execute JD matching task"""
    try:
        # Direct tool call instead of complex crew setup
        result = jd_match_tool_crewai.func(resume_text, job_description)
        return result
    except Exception as e:
        return f"ERROR in JD analysis: {str(e)}\n{traceback.format_exc()}"

def crew_run_insight(insight_agent, resume_text, jd_result):
    """Execute insight generation task"""
    try:
        # Direct tool call instead of complex crew setup
        result = critical_insight_tool_crewai.func(resume_text, jd_result)
        return result
    except Exception as e:
        return f"ERROR in insight generation: {str(e)}\n{traceback.format_exc()}"

def crew_run_hr(hr_agent, question, context):
    """Execute HR chat task"""
    try:
        # Direct tool call instead of complex crew setup
        result = hr_chat_tool_crewai.func(question, context)
        return result
    except Exception as e:
        return f"ERROR in HR chat: {str(e)}\n{traceback.format_exc()}"

In [ ]:
# Gradio callbacks
def cb_set_model(tag):
    try:
        _ensure_ollama_ready()
        _ensure_model_available(tag or DEFAULT_MODEL)
        _probe_chat_support()
        return f"✅ Active model: {ACTIVE_MODEL}"
    except Exception as e:
        return f"❌ {e}"

def cb_analyze_resume(resume_file, job_description, selected_model):
    try:
        _ensure_ollama_ready()
        _ensure_model_available(selected_model or DEFAULT_MODEL)
        _probe_chat_support()
    except Exception as e:
        return f"❌ {e}", "", ""

    if not resume_file or not job_description or not str(job_description).strip():
        return "❌ Please upload a resume and provide a job description.", "", ""

    resume_text = extract_text_from_uploaded_file(resume_file)
    if resume_text.startswith("❌ ERROR"):
        return resume_text, "File processing failed.", ""

    try:
        jd_agent, insight_agent, hr_agent = build_crewai_agents()
        jd_res = crew_run_jd(jd_agent, resume_text, job_description)

        if jd_res.startswith("ERROR"):
            return jd_res, "Analysis failed.", ""

        insights = crew_run_insight(insight_agent, resume_text, jd_res)

        if insights.startswith("ERROR"):
            return f"{jd_res}\n\nInsight generation failed: {insights}", "Partial analysis complete.", ""

        global CURRENT_CONTEXT, LAST_ANALYSIS
        combined = f"JD MATCHING ANALYSIS:\n{jd_res}\n\nCRITICAL INSIGHTS:\n{insights}"
        CURRENT_CONTEXT = limit_input(combined, MAX_CONTEXT_TOKENS)
        LAST_ANALYSIS = {"jd": jd_res, "insights": insights}

        resume_tokens = len(resume_text.split())
        jd_tokens = len(str(job_description).split())
        out = (
            f"🤖 CrewAI Agents | 🦙 Model: {ACTIVE_MODEL}\n"
            f"📊 (Resume: {resume_tokens} tokens, JD: {jd_tokens} tokens)\n\n"
            f"📊 JD MATCHING ANALYSIS:\n{jd_res}\n\n"
            f"📝 CRITICAL INSIGHTS:\n{insights}"
        )
        return out, "✅ Analysis complete via CrewAI agents. You can now use the HR Chat tab.", ""
    except Exception as e:
        return f"❌ CRITICAL ERROR: {e}\n{traceback.format_exc()}", "", ""

def cb_hr_chat(question, selected_model):
    if not CURRENT_CONTEXT:
        return "⚠️ Please analyze a candidate first in the Resume Analysis tab."
    if not question or not str(question).strip():
        return "Please ask a question about the candidate."
    try:
        _ensure_ollama_ready()
        _ensure_model_available(selected_model or DEFAULT_MODEL)
        _probe_chat_support()
        jd_agent, insight_agent, hr_agent = build_crewai_agents()
        return crew_run_hr(hr_agent, question, CURRENT_CONTEXT)
    except Exception as e:
        return f"❌ Error: {e}"

def cb_batch_rank(resume_files, job_description, selected_model):
    try:
        _ensure_ollama_ready()
        _ensure_model_available(selected_model or DEFAULT_MODEL)
        _probe_chat_support()
    except Exception as e:
        return f"❌ {e}", ""

    if not resume_files or len(resume_files) == 0:
        return "❌ Please upload at least 2 resume files.", ""
    if len(resume_files) < 2:
        return "❌ Please upload at least 2 resumes for comparison.", ""
    if len(resume_files) > 5:
        return "❌ Maximum 5 resumes allowed.", ""
    if not job_description or not str(job_description).strip():
        return "❌ Please provide a job description.", ""

    jd_agent, insight_agent, hr_agent = build_crewai_agents()

    results = []
    for i, f in enumerate(resume_files, 1):
        try:
            text = extract_text_from_uploaded_file(f)
            name = getattr(f, "name", f"Resume_{i}").split("\\")[-1].split("/")[-1]
            if text.startswith("❌ ERROR"):
                results.append({"rank": None, "file_name": name, "status": "failed", "score": 0, "jd": text, "insights": ""})
                continue
            jd_res = crew_run_jd(jd_agent, text, job_description)
            if jd_res.startswith("ERROR"):
                results.append({"rank": None, "file_name": name, "status": "failed", "score": 0, "jd": jd_res, "insights": ""})
                continue
            score = extract_match_score(jd_res)
            insights = crew_run_insight(insight_agent, text, jd_res)
            results.append({"rank": None, "file_name": name, "status": "success", "score": score, "jd": jd_res, "insights": insights})
        except Exception as e:
            name = getattr(f, "name", f"Resume_{i}").split("\\")[-1].split("/")[-1]
            results.append({"rank": None, "file_name": name, "status": "failed", "score": 0, "jd": f"❌ ERROR: {e}", "insights": ""})

    results.sort(key=lambda x: x["score"], reverse=True)
    for rnk, item in enumerate(results, 1):
        item["rank"] = rnk

    summary = (
        "🏆 CANDIDATE RANKING SUMMARY\n"
        f"🤖 CrewAI Agents | 🦙 Model: {ACTIVE_MODEL}\n"
        f"📊 Total Processed: {len(resume_files)}\n"
        f"✅ Successfully Analyzed: {sum(1 for r in results if r['status']=='success')}\n"
        f"❌ Failed: {sum(1 for r in results if r['status']=='failed')}\n"
    )

    details = "🏆 DETAILED CANDIDATE RANKINGS\n\n"
    for c in results:
        emoji = "🥇" if c["rank"] == 1 else "🥈" if c["rank"] == 2 else "🥉" if c["rank"] == 3 else f"{c['rank']}️⃣"
        if c["score"] >= 85:
            stars = "⭐⭐⭐⭐⭐"; rec = "STRONG RECOMMEND"
        elif c["score"] >= 70:
            stars = "⭐⭐⭐⭐"; rec = "RECOMMEND"
        elif c["score"] >= 50:
            stars = "⭐⭐⭐"; rec = "CAUTIOUS YES"
        elif c["score"] >= 30:
            stars = "⭐⭐"; rec = "MAYBE"
        else:
            stars = "⭐"; rec = "HARD PASS"
        details += (
            f"{'='*80}\n"
            f"{emoji} RANK #{c['rank']} - {c['file_name']}\n"
            f"{'='*80}\n"
            f"📊 MATCH SCORE: {c['score']}/100 {stars}\n"
            f"💼 RECOMMENDATION: {rec}\n\n"
        )
        if c["status"] == "success":
            details += f"📄 ANALYSIS:\n{c['jd']}\n\n📝 INSIGHTS:\n{c['insights']}\n\n"
        else:
            details += f"❌ ERROR: {c['jd']}\n\n"

    return summary, details

In [ ]:
# UI
def build_ui():
    with gr.Blocks(title="Recruitment Portal (CrewAI + Ollama)", theme="soft", analytics_enabled=False) as demo:
        gr.Markdown(
            f"""
# 🤖 CrewAI Recruitment Portal (Fixed)
## Multi-agent pipeline with local Ollama models

- Agents: JD Matcher, Insight Generator, HR Chat
- Default model: {DEFAULT_MODEL}
- **Fixed**: LiteLLM errors, CrewAI tool integration, task execution
            """.strip()
        )

        with gr.Row():
            model_select = gr.Dropdown(choices=MODEL_CHOICES, value=DEFAULT_MODEL, label="Active Model")
            model_status = gr.Textbox(label="Model Status", interactive=False, value="Ready")
        model_select.change(cb_set_model, inputs=[model_select], outputs=[model_status])

        with gr.Tab("📄 Resume Analysis"):
            with gr.Row():
                with gr.Column(scale=1):
                    resume_upload = gr.File(file_types=[".pdf", ".docx", ".txt"], label="Resume File (PDF/DOCX/TXT)")
                    job_description = gr.Textbox(lines=8, placeholder="Paste the job description here...", label="Job Description")
                    analyze_button = gr.Button("🤖 Analyze with CrewAI", variant="primary")
                with gr.Column(scale=2):
                    analysis_output = gr.Textbox(lines=22, label="Analysis Output", interactive=False)
                    status_message = gr.Textbox(label="Status", interactive=False, value="Ready")
            analyze_button.click(
                cb_analyze_resume,
                inputs=[resume_upload, job_description, model_select],
                outputs=[analysis_output, status_message, gr.Textbox(visible=False)]
            )

        with gr.Tab("💬 HR Chat Assistant"):
            gr.Markdown("Ask questions about the analyzed candidate ONLY")
            with gr.Row():
                with gr.Column():
                    hr_question = gr.Textbox(
                        placeholder="Ask about hiring decision, risks, score rationale, interview focus...",
                        label=f"Your Question (Max {MAX_QUESTION_TOKENS} tokens)",
                        lines=3
                    )
                    chat_button = gr.Button("💬 Ask")
                with gr.Column():
                    chat_response = gr.Textbox(lines=14, label=f"Response (Max {MAX_OUTPUT_TOKENS} tokens)", interactive=False)
            chat_button.click(cb_hr_chat, inputs=[hr_question, model_select], outputs=[chat_response])

        with gr.Tab("📊 Batch Resume Ranking"):
            with gr.Row():
                with gr.Column(scale=1):
                    batch_files = gr.File(file_count="multiple", file_types=[".pdf", ".docx", ".txt"], label="Upload Resumes (2-5 files)")
                    batch_jd = gr.Textbox(lines=8, placeholder="Paste the job description here...", label="Job Description (Same for All)")
                    batch_btn = gr.Button("🏆 Rank Candidates", variant="primary")
                with gr.Column(scale=2):
                    summary_box = gr.Textbox(lines=8, label="Summary", interactive=False)
                    ranking_box = gr.Textbox(lines=22, label="Detailed Rankings", interactive=False)
            batch_btn.click(cb_batch_rank, inputs=[batch_files, batch_jd, model_select], outputs=[summary_box, ranking_box])
    return demo

In [ ]:
# Launch (notebook-safe)
def launch_app():
    try:
        loop = asyncio.get_event_loop()
        if loop.is_closed():
            asyncio.set_event_loop(asyncio.new_event_loop())
        nest_asyncio.apply()
    except Exception:
        pass

    gr.close_all()
    demo = build_ui()

    in_colab = "COLAB_RELEASE_TAG" in os.environ or "COLAB_GPU" in os.environ

    demo.queue()
    demo.launch(
        server_name="0.0.0.0",
        share=True if in_colab else False,
        inline=True,
        debug=False,
        prevent_thread_lock=True,
        show_error=True
    )

if __name__ == "__main__":
    _ensure_ollama_ready()
    _ensure_model_available(DEFAULT_MODEL)
    _probe_chat_support()
    print(f"✅ Ready. Active model: {ACTIVE_MODEL} | Chat endpoint: {('yes' if CHAT_AVAILABLE else 'fallback')}")
    launch_app()